In [5]:
from bs4 import BeautifulSoup
import requests
import selenium
import random
from time import sleep
import pandas as pd

In [6]:
# logging is important
logger = logging.getLogger()
dstamp = datetime.now().strftime('%m_%d')
fhandler = logging.FileHandler(filename= dstamp + 'session.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)


# Constants

In [9]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0'} 

In [10]:
baseurl =  'https://data.gov.ph/sites/default/files/'

In [14]:
def urlToBs(url, headers):
    sleep(random.uniform(0,0.5))
    pg = requests.get(url, headers)
    sleep(1)
    bs = BeautifulSoup(pg.content, 'html.parser')
    return bs

In [31]:
def sp_to_lturl(bsPage):
    return [a['href'] for a in bsPage.find_all("a", {'class':"btn btn-primary data-link"})] 

In [32]:
purl =  'https://data.gov.ph/?q=dataset/climatological-normal-values'

In [21]:
bsPage = urlToBs(purl, headers)

In [33]:
hturllt = sp_to_lturl(bsPage) 

In [45]:
def fileOut(content, fname):
    fout = open(fname, 'wb')
    fout.write(content)
    fout.close()

In [47]:
r = requests.get(hturllt[0], allow_redirects=True)

In [52]:
hturllt[0].split('/')[-1]

'pagasanormvalalabatquez1999-2010.csv'

In [55]:
for url in hturllt:
    r = requests.get(url, allow_redirects=True)
    fileOut(r.content, './precipCSV/' + url.split('/')[-1])

In [79]:
def readAllCsv(lt, fp):
    dtdfImported = {}
    for fname in lt:
        logging.info(fname)
        dtdfImported[fname] = pd.read_csv(fp + fname)
        locstr = fname.split('.')[0][13:] 
        dtdfImported[fname]['location'] = locstr
    return dtdfImported

In [65]:
ltfname = [url.split('/')[-1] for url in hturllt]

In [86]:
dtdfImported =  readAllCsv(ltfname, './precipCSV/')

In [87]:
len('pagasanormval')

13

In [89]:
ltdfImported = [df for df in  dtdfImported.values()]

In [90]:
dfFinal = pd.concat(ltdfImported)

In [93]:
dfFinal.shape

(715, 23)

In [94]:
dfFinal.columns

Index(['month', 'rainfall_amount', 'rainfall_number_of_rd', 'temp_max',
       'temp_min', 'temp_mean', 'temp_dry_bulb', 'temp_wet_bulb',
       'temp_dew_pt', 'vapor_pressure_mbs', 'rel_hum_percent', 'mslp_mbs',
       'wind_dir_16pt', 'wind_spd_mps', 'cloud_amount_okta',
       'number_of_days_with_tstm', 'number_of_days_with_ltng', 'location',
       'rel_hum', 'mslp', 'vapor_pressure', ' ', 'wind_spd'],
      dtype='object')

In [95]:
def pickleOut(obj, fname):
    fout =  open(fname, 'wb')
    pickle.dump(obj, fout)
    fout.close()

In [96]:
pickleOut(dfFinal, 'dfprecip.pkl')